In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
###############################################
#################AAAAAAAAAAAAAAAAAAAAA###############3
%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
#导入csv并标准化json编码
def load_df(csv_path='D:/zhiliao/kaggle/predict_income/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df


In [7]:
%%time

test = load_df(csv_path='D:/zhiliao/kaggle/predict_income/test.csv')
train2 = load_df()


KeyboardInterrupt



In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
train2['revenue_status']=train2['totals.transactionRevenue'].apply(lambda x: 0 if x==0 else 1)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for train_index, stack_index in split.split(train2,train2['revenue_status']):               
    train = train2.loc[train_index]             
    train1  = test.loc[stack_index]
    

def get_folds(df=None, n_splits=5):
    """Returns dataframe indices corresponding to fullVisitorId Group KFold"""
    # Get sorted unique visitors
    unique_vis = np.array(sorted(df['fullVisitorId'].unique()))

    # Get folds
    folds = GroupKFold(n_splits=n_splits)
    fold_ids = []
    ids = np.arange(df.shape[0])
    for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
        fold_ids.append(
            [
                ids[df['fullVisitorId'].isin(unique_vis[trn_vis])],
                ids[df['fullVisitorId'].isin(unique_vis[val_vis])]
            ]
        )
    return fold_ids

E:\anaconda\envs\tenseflow36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



In [6]:
train.shape

(451826, 56)

In [6]:
y_reg = train['totals.transactionRevenue'].fillna(0)
del train['totals.transactionRevenue']

if 'totals.transactionRevenue' in test.columns:
    del test['totals.transactionRevenue']

In [7]:
for df in [train, test]:
    df['date'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['sess_date_dow'] = df['date'].dt.dayofweek
    df['sess_date_hours'] = df['date'].dt.hour
    df['sess_date_dom'] = df['date'].dt.day
    

In [8]:
excluded_features = [
    'date', 'fullVisitorId', 'sessionId', 'totals.transactionRevenue', 
    'visitId', 'visitStartTime','trafficSource.campaignCode'
]

categorical_features = [
    _f for _f in train.columns
    if (_f not in excluded_features) & (train[_f].dtype == 'object')
]

In [9]:
for f in categorical_features:
    print(f)
    train[f], indexer = pd.factorize(train[f])
    test[f] = indexer.get_indexer(test[f])

channelGrouping
socialEngagementType
device.browser
device.browserSize
device.browserVersion
device.deviceCategory
device.flashVersion
device.language
device.mobileDeviceBranding
device.mobileDeviceInfo
device.mobileDeviceMarketingName
device.mobileDeviceModel
device.mobileInputSelector
device.operatingSystem
device.operatingSystemVersion
device.screenColors
device.screenResolution
geoNetwork.city
geoNetwork.cityId
geoNetwork.continent
geoNetwork.country
geoNetwork.latitude
geoNetwork.longitude
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.networkLocation
geoNetwork.region
geoNetwork.subContinent
totals.bounces
totals.hits
totals.newVisits
totals.pageviews
totals.visits
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.criteriaParameters
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.isTrueDi

In [9]:
train

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,sess_date_dow,sess_date_hours,sess_date_dom
0,0,2016-09-02 15:33:05,1131660440785968503,1131660440785968503_1472830385,0,1472830385,1,1472830385,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,4,15,2
1,0,2016-09-03 05:22:27,377306020877927890,377306020877927890_1472880147,0,1472880147,1,1472880147,1,0,0,0,0,False,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,5,5,3
2,0,2016-09-03 01:16:26,3895546263509774583,3895546263509774583_1472865386,0,1472865386,1,1472865386,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,2,0,2,2,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,5,1,3
3,0,2016-09-03 05:40:13,4763447161404445595,4763447161404445595_1472881213,0,1472881213,1,1472881213,2,0,0,0,0,False,0,0,0,0,0,0,2,0,0,0,1,0,0,3,0,0,1,2,0,1,3,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,1,0,-1,0,5,5,3
4,0,2016-09-02 13:23:20,27294437909732085,27294437909732085_1472822600,0,1472822600,2,1472822600,0,0,0,1,0,True,0,0,0,0,0,0,3,0,0,0,1,0,2,4,0,0,1,2,0,1,4,0,0,-1,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,0,0,0,-1,0,4,13,2
5,0,2016-09-02 09:06:34,2938943183656635653,2938943183656635653_1472807194,0,1472807194,1,1472807194,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,1,0,2,5,0,0,1,3,0,1,2,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,4,9,2
6,0,2016-09-02 11:54:01,1905672039242460897,1905672039242460897_1472817241,0,1472817241,1,1472817241,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,1,0,0,6,0,0,1,2,0,1,5,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,4,11,2
7,0,2016-09-02 10:36:42,537222803633850821,537222803633850821_1472812602,0,1472812602,1,1472812602,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,3,0,1,1,0,0,0,4,0,3,1,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,4,10,2
8,0,2016-09-02 08:43:04,4445454811831400414,4445454811831400414_1472805784,0,1472805784,1,1472805784,3,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,1,0,2,7,0,0,1,5,0,1,6,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,4,8,2
9,0,2016-09-02 10:31:12,9499785259412240342,9499785259412240342_1472812272,0,1472812272,1,1472812272,1,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,1,0,2,8,0,0,1,6,0,1,6,0,0,0,0,0,-1,-1,0,-1,-1,-1,-1,0,NaN,-1,0,0,-1,0,4,10,2


In [10]:
from sklearn.model_selection import GroupKFold
folds = get_folds(df=train, n_splits=5)
""""[[array([     1,      2,      3, ..., 903650, 903651, 903652]),
  array([     0,      9,     14, ..., 903600, 903611, 903648])],
 [array([     0,      1,      5, ..., 903650, 903651, 903652]),
  array([     2,      3,      4, ..., 903639, 903642, 903645])],
 [array([     0,      1,      2, ..., 903650, 903651, 903652]),
  array([     5,      8,     12, ..., 903641, 903644, 903649])],
 [array([     0,      1,      2, ..., 903650, 903651, 903652]),
  array([     7,     10,     24, ..., 903636, 903637, 903646])],
 [array([     0,      2,      3, ..., 903646, 903648, 903649]),
  array([     1,     11,     13, ..., 903650, 903651, 903652])]]"""

'"[[array([     1,      2,      3, ..., 903650, 903651, 903652]),\n  array([     0,      9,     14, ..., 903600, 903611, 903648])],\n [array([     0,      1,      5, ..., 903650, 903651, 903652]),\n  array([     2,      3,      4, ..., 903639, 903642, 903645])],\n [array([     0,      1,      2, ..., 903650, 903651, 903652]),\n  array([     5,      8,     12, ..., 903641, 903644, 903649])],\n [array([     0,      1,      2, ..., 903650, 903651, 903652]),\n  array([     7,     10,     24, ..., 903636, 903637, 903646])],\n [array([     0,      2,      3, ..., 903646, 903648, 903649]),\n  array([     1,     11,     13, ..., 903650, 903651, 903652])]]'

In [11]:
 
#未解之谜
# for col in train.columns:  
#     if (col not in excluded_features)&(train[col].dtype=='int'):
#         train[col]=train[col].astype(float)
from sklearn.metrics import mean_squared_error

for col in train.columns:
    if (col not in excluded_features)&(train[col].dtype=='int64'):
        train[col]=train[col].astype(float)
y_reg=y_reg.astype(float)        
a=['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'trafficSource.adContent',
       'trafficSource.campaign', 'trafficSource.isTrueDirect',
       'trafficSource.keyword', 'trafficSource.medium',
       'trafficSource.referralPath', 'trafficSource.source']
train_features = [_f for _f in a if _f not in excluded_features]
print(train_features)
importances = pd.DataFrame()
oof_reg_preds = np.zeros(train.shape[0])
sub_reg_preds = np.zeros(test.shape[0])
for fold_, (trn_, val_) in enumerate(folds):
    
    trn_x, trn_y = train[train_features].iloc[trn_], y_reg.iloc[trn_]
    val_x, val_y = train[train_features].iloc[val_], y_reg.iloc[val_]
#     trn_y=trn_y.astype(float)
#     val_y=val_y.astype(float)
    reg = lgb.LGBMRegressor(
        num_leaves=31,
        learning_rate=0.03,
        n_estimators=1000,
        subsample=.9,
        colsample_bytree=.9,
        random_state=1
    )
    reg.fit(
        trn_x, np.log1p(trn_y),
        eval_set=[(val_x, np.log1p(val_y))],
        early_stopping_rounds=50,
        verbose=100,
        eval_metric='rmse'
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = train_features
    imp_df['gain'] = reg.booster_.feature_importance(importance_type='gain')
    
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_reg_preds[val_] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    oof_reg_preds[oof_reg_preds < 0] = 0
    _preds = reg.predict(test[train_features], num_iteration=reg.best_iteration_)
    _preds[_preds < 0] = 0
    sub_reg_preds += np.expm1(_preds) # len(folds)#每一折都加了一次，相当于取平均
    
mean_squared_error(np.log1p(y_reg), oof_reg_preds) ** .5

['channelGrouping', 'visitNumber', 'device.browser', 'device.deviceCategory', 'device.isMobile', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 'trafficSource.adContent', 'trafficSource.campaign', 'trafficSource.isTrueDirect', 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source']
Training until validation scores don't improve for 50 rounds.
[100]	valid_0's l2: 2.77094	valid_0's rmse: 1.66461
[200]	valid_0's l2: 2.72705	valid_0's rmse: 1.65138
[300]	valid_0's l2: 2.71546	valid_0's rmse: 1.64786
[400]	valid_0's l2: 2.70909	valid_0's rmse: 1.64593
[500]	valid_0's l2: 2.70903	valid_0's rmse: 1.64591
Early stopping, best iteration is:
[475]	valid_0's l2: 2.70786	valid_0's rmse: 1.64556
Training until validation scores don't improve f

1.6398588603495432

In [35]:
train['predictions'] = np.expm1(oof_reg_preds)
test['predictions'] = sub_reg_preds
trn_data = train[train_features + ['fullVisitorId']].groupby('fullVisitorId').mean()

In [29]:
trn_data

,channelGrouping,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,trafficSource.adContent,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
fullVisitorId,,,,,,,,,,,,,,,,,,,,,,,,
0000010278554503158,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,69.0,1.0,400.0,1.0,1.0,-1.0,10.0,0.0,7.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000020424342248747,0.0,1.0,0.0,0.0,0.0,0.0,46.0,3.0,42.0,0.0,6731.0,38.0,7.0,-1.0,16.0,0.0,12.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000027376579751715,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,33.0,1.0,91.0,1.0,10.0,-1.0,5.0,0.0,4.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000039460501403861,6.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,134.0,1.0,7.0,-1.0,1.0,0.0,1.0,-1.0,0.0,-1.0,-1.0,1.0,72.0,25.0
0000040862739425590,2.0,1.5,0.0,0.0,0.0,1.0,111.0,3.0,33.0,3.0,110.0,22.0,10.0,-1.0,1.5,-0.5,1.5,-1.0,1.0,-0.5,13.0,2.0,-1.0,0.0
000005103959234087,0.0,1.0,0.0,1.0,1.0,3.0,1.0,3.0,33.0,1.0,91.0,1.0,10.0,-1.0,9.0,0.0,7.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000062267706107999,3.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,2.0,1.0,7.0,0.0,0.0,0.0,0.0,-1.0,2.0,-1.0,-1.0,3.0,-1.0,7.0
0000068403966359845,0.0,1.0,4.0,2.0,1.0,4.0,33.0,3.0,33.0,6.0,1170.0,22.0,10.0,-1.0,1.0,0.0,1.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000085059828173212,4.0,1.0,4.0,1.0,1.0,4.0,42.0,3.0,33.0,8.0,23.0,35.0,10.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,4.0,-1.0,8.0


In [36]:

trn_data['visitNumber']=train[['visitNumber']+['fullVisitorId']].groupby('fullVisitorId').agg('max')
trn_data['totals.bounces']=train[['totals.bounces']+['fullVisitorId']].groupby('fullVisitorId').agg('sum')
trn_data['totals.hits']=train[['totals.hits']+['fullVisitorId']].groupby('fullVisitorId').agg('sum')
trn_data['totals.newVisits']=train[['totals.newVisits']+['fullVisitorId']].groupby('fullVisitorId').agg('min')
trn_data['totals.pageviews']=train[['totals.pageviews']+['fullVisitorId']].groupby('fullVisitorId').agg('sum')


In [27]:
trn_data

,channelGrouping,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,trafficSource.adContent,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
fullVisitorId,,,,,,,,,,,,,,,,,,,,,,,,
0000010278554503158,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,69.0,1.0,400.0,1.0,1.0,-1.0,10.0,0.0,7.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000020424342248747,0.0,1.0,0.0,0.0,0.0,0.0,46.0,3.0,42.0,0.0,6731.0,38.0,7.0,-1.0,16.0,0.0,12.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000027376579751715,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,33.0,1.0,91.0,1.0,10.0,-1.0,5.0,0.0,4.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000039460501403861,6.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,134.0,1.0,7.0,-1.0,1.0,0.0,1.0,-1.0,0.0,-1.0,-1.0,1.0,72.0,25.0
0000040862739425590,2.0,2.0,0.0,0.0,0.0,1.0,111.0,3.0,33.0,3.0,110.0,22.0,10.0,-2.0,3.0,-1.0,3.0,-1.0,1.0,-0.5,13.0,2.0,-1.0,0.0
000005103959234087,0.0,1.0,0.0,1.0,1.0,3.0,1.0,3.0,33.0,1.0,91.0,1.0,10.0,-1.0,9.0,0.0,7.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000062267706107999,3.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,2.0,1.0,7.0,0.0,0.0,0.0,0.0,-1.0,2.0,-1.0,-1.0,3.0,-1.0,7.0
0000068403966359845,0.0,1.0,4.0,2.0,1.0,4.0,33.0,3.0,33.0,6.0,1170.0,22.0,10.0,-1.0,1.0,0.0,1.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,0.0
0000085059828173212,4.0,1.0,4.0,1.0,1.0,4.0,42.0,3.0,33.0,8.0,23.0,35.0,10.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,4.0,-1.0,8.0


In [27]:
train_features

['channelGrouping',
 'visitNumber',
 'device.browser',
 'device.deviceCategory',
 'device.isMobile',
 'device.operatingSystem',
 'geoNetwork.city',
 'geoNetwork.continent',
 'geoNetwork.country',
 'geoNetwork.metro',
 'geoNetwork.networkDomain',
 'geoNetwork.region',
 'geoNetwork.subContinent',
 'totals.bounces',
 'totals.hits',
 'totals.newVisits',
 'totals.pageviews',
 'trafficSource.adContent',
 'trafficSource.campaign',
 'trafficSource.isTrueDirect',
 'trafficSource.keyword',
 'trafficSource.medium',
 'trafficSource.referralPath',
 'trafficSource.source']

In [37]:

# Create a list of predictions for each Visitor
trn_pred_list = train[['fullVisitorId', 'predictions']].groupby('fullVisitorId')\
    .apply(lambda df: list(df.predictions))\
    .apply(lambda x: {'pred_'+str(i): pred for i, pred in enumerate(x)})

In [ ]:
##将一个ID访问了第几次作为特征

In [38]:
%%time
# Create a DataFrame with VisitorId as index
# trn_pred_list contains dict 
# so creating a dataframe from it will expand dict values into columns
import gc
trn_all_predictions = pd.DataFrame(list(trn_pred_list.values), index=trn_data.index)#trn_data index 就是一列ID
trn_feats = trn_all_predictions.columns
trn_all_predictions['t_mean'] = np.log1p(trn_all_predictions[trn_feats].mean(axis=1))
print(1)
trn_all_predictions['t_median'] = np.log1p(trn_all_predictions[trn_feats].median(axis=1))
print(2)
trn_all_predictions['t_sum_log'] = np.log1p(trn_all_predictions[trn_feats]).sum(axis=1)
print(1)
trn_all_predictions['t_sum_act'] = np.log1p(trn_all_predictions[trn_feats].fillna(0).sum(axis=1))
print(2)
trn_all_predictions['t_nb_sess'] = trn_all_predictions[trn_feats].isnull().sum(axis=1)
print(1)
# full_data = pd.concat([trn_data, trn_all_predictions], axis=1)
full_data = trn_all_predictions

1
2


E:\anaconda\envs\tenseflow36\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning:

invalid value encountered in log1p



1
2
1
Wall time: 3min 14s


In [34]:
del trn_data, trn_all_predictions
gc.collect()
full_data.shape

NameError: name 'trn_all_predictions' is not defined

In [14]:
trn_feats

Index(['pred_0', 'pred_1', 'pred_10', 'pred_100', 'pred_101', 'pred_102',
       'pred_103', 'pred_104', 'pred_105', 'pred_106',
       ...
       'pred_90', 'pred_91', 'pred_92', 'pred_93', 'pred_94', 'pred_95',
       'pred_96', 'pred_97', 'pred_98', 'pred_99'],
      dtype='object', length=278)

In [19]:
%%time
sub_pred_list = test[['fullVisitorId', 'predictions']].groupby('fullVisitorId')\
    .apply(lambda df: list(df.predictions))\
    .apply(lambda x: {'pred_'+str(i): pred for i, pred in enumerate(x)})

Wall time: 24.8 s


In [20]:
%%time
sub_data = test[train_features + ['fullVisitorId']].groupby('fullVisitorId').mean()

sub_data['visitNumber']=test[['visitNumber']+['fullVisitorId']].groupby('fullVisitorId').agg('max')
sub_data['totals.bounces']=test[['totals.bounces']+['fullVisitorId']].groupby('fullVisitorId').agg('sum')
sub_data['totals.hits']=test[['totals.hits']+['fullVisitorId']].groupby('fullVisitorId').agg('sum')
sub_data['totals.newVisits']=test[['totals.newVisits']+['fullVisitorId']].groupby('fullVisitorId').agg('min')
sub_data['totals.pageviews']=test[['totals.pageviews']+['fullVisitorId']].groupby('fullVisitorId').agg('sum')
sub_all_predictions = pd.DataFrame(list(sub_pred_list.values), index=sub_data.index)
for f in trn_feats:
    if f not in sub_all_predictions.columns:
        sub_all_predictions[f] = np.nan
sub_all_predictions['t_mean'] = np.log1p(sub_all_predictions[trn_feats].mean(axis=1))
sub_all_predictions['t_median'] = np.log1p(sub_all_predictions[trn_feats].median(axis=1))
sub_all_predictions['t_sum_log'] = np.log1p(sub_all_predictions[trn_feats]).sum(axis=1)
sub_all_predictions['t_sum_act'] = np.log1p(sub_all_predictions[trn_feats].fillna(0).sum(axis=1))
sub_all_predictions['t_nb_sess'] = sub_all_predictions[trn_feats].isnull().sum(axis=1)
sub_full_data = pd.concat([sub_data, sub_all_predictions], axis=1)
# sub_full_data=sub_all_predictions
del sub_data, sub_all_predictions
gc.collect()
sub_full_data.shape

E:\anaconda\envs\tenseflow36\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning:

invalid value encountered in log1p



Wall time: 1min 1s


In [21]:
train['target'] = y_reg
trn_user_target = train[['fullVisitorId', 'target']].groupby('fullVisitorId').sum()

In [22]:
full_data[['pred_1']].index

Index(['0000010278554503158', '0000020424342248747', '0000027376579751715',
       '0000039460501403861', '0000040862739425590', '000005103959234087',
       '0000062267706107999', '0000068403966359845', '0000085059828173212',
       '0000093957001069502',
       ...
       '9999825631375939274', '9999887420016307570', '9999906724458612544',
       '9999944980984230465', '9999955365228667450', '9999963186378918199',
       '999997225970956660', '999997550040396460', '9999978264901065827',
       '9999986437109498564'],
      dtype='object', name='fullVisitorId', length=714167)

In [23]:
%%time
folds = get_folds(df=full_data[['pred_1']].reset_index(), n_splits=5)

oof_preds = np.zeros(full_data.shape[0])
sub_preds = np.zeros(sub_full_data.shape[0])
vis_importances = pd.DataFrame()

for fold_, (trn_, val_) in enumerate(folds):
    trn_x, trn_y = full_data.iloc[trn_], trn_user_target['target'].iloc[trn_]
    val_x, val_y = full_data.iloc[val_], trn_user_target['target'].iloc[val_]
    
    reg = lgb.LGBMRegressor(
        num_leaves=31,
        learning_rate=0.03,
        n_estimators=1000,
        subsample=.9,
        colsample_bytree=.9,
        random_state=1
    )
    reg.fit(
        trn_x, np.log1p(trn_y),
        eval_set=[(trn_x, np.log1p(trn_y)), (val_x, np.log1p(val_y))],
        eval_names=['TRAIN', 'VALID'],
        early_stopping_rounds=50,
        eval_metric='rmse',
        verbose=100
    )
    
    imp_df = pd.DataFrame()
    imp_df['feature'] = trn_x.columns
    imp_df['gain'] = reg.booster_.feature_importance(importance_type='gain')
    
    imp_df['fold'] = fold_ + 1
    vis_importances = pd.concat([vis_importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    oof_preds[oof_preds < 0] = 0
        # Make sure features are in the same order
    _preds = reg.predict(sub_full_data[full_data.columns], num_iteration=reg.best_iteration_)
    _preds[_preds < 0] = 0
    sub_preds += _preds / len(folds)
    
mean_squared_error(np.log1p(trn_user_target['target']), oof_preds) ** .5

Training until validation scores don't improve for 50 rounds.
[100]	TRAIN's l2: 2.4531	TRAIN's rmse: 1.56624	VALID's l2: 2.56634	VALID's rmse: 1.60198
Early stopping, best iteration is:
[134]	TRAIN's l2: 2.42018	TRAIN's rmse: 1.55569	VALID's l2: 2.56193	VALID's rmse: 1.6006
Training until validation scores don't improve for 50 rounds.
[100]	TRAIN's l2: 2.4476	TRAIN's rmse: 1.56448	VALID's l2: 2.58971	VALID's rmse: 1.60926
Early stopping, best iteration is:
[111]	TRAIN's l2: 2.4365	TRAIN's rmse: 1.56093	VALID's l2: 2.58701	VALID's rmse: 1.60842
Training until validation scores don't improve for 50 rounds.
[100]	TRAIN's l2: 2.44482	TRAIN's rmse: 1.56359	VALID's l2: 2.596	VALID's rmse: 1.61121
Early stopping, best iteration is:
[134]	TRAIN's l2: 2.41269	TRAIN's rmse: 1.55328	VALID's l2: 2.59355	VALID's rmse: 1.61045
Training until validation scores don't improve for 50 rounds.
[100]	TRAIN's l2: 2.46564	TRAIN's rmse: 1.57024	VALID's l2: 2.49556	VALID's rmse: 1.57974
Early stopping, best it

In [24]:
sub_full_data['PredictedLogRevenue'] = sub_preds
sub_full_data[['PredictedLogRevenue']].to_csv('new_test3.csv', index=True)

In [34]:
sub_full_data['PredictedLogRevenue']

fullVisitorId
0000000259678714014     0.934184
0000049363351866189     0.011409
0000053049821714864     0.008445
0000059488412965267     0.011409
0000085840370633780     0.008445
0000091131414287111     0.008445
0000117255350596610     8.555077
0000118334805178127     0.008445
0000130646294093000     0.008445
0000150005271820273     0.008445
0000166374699289385     0.014789
0000174453501096099     0.008445
000018122977590134      9.124822
000020731284570628      0.008445
0000232022622082281     0.008445
0000271086753662651     0.008445
0000282648818935742     0.008445
00003038793126460       0.008445
0000313524203455157     0.008445
0000324924635296742     0.008693
0000338548677636278     0.008445
0000354865008116989     0.008445
0000359974620542953     0.016153
0000384434116640351     6.925070
0000385653946068037     0.008445
0000397214032106948     0.008445
0000414003317636552     5.179237
0000421850492821864     0.008445
0000436683523507380     0.023551
0000443488189725694     0.008

In [33]:
sub_full_data[['PredictedLogRevenue']]

,PredictedLogRevenue
fullVisitorId,
0000000259678714014,0.934184
0000049363351866189,0.011409
0000053049821714864,0.008445
0000059488412965267,0.011409
0000085840370633780,0.008445
0000091131414287111,0.008445
0000117255350596610,8.555077
0000118334805178127,0.008445
0000130646294093000,0.008445


In [35]:
sub_full_data

,channelGrouping,socialEngagementType,visitNumber,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,sess_date_dow,sess_date_hours,sess_date_dom,pred_0,pred_1,pred_10,pred_100,pred_101,pred_102,pred_103,pred_104,pred_105,pred_106,pred_107,pred_108,pred_109,pred_11,pred_110,pred_111,pred_112,pred_113,pred_114,pred_115,pred_116,pred_117,pred_118,pred_119,pred_12,pred_120,pred_121,pred_122,pred_123,pred_124,pred_125,pred_126,pred_127,pred_128,pred_129,pred_13,pred_130,pred_131,pred_132,pred_133,pred_134,pred_135,pred_136,pred_137,pred_138,pred_139,pred_14,pred_140,pred_141,pred_142,pred_143,pred_144,pred_145,pred_146,pred_147,pred_148,pred_149,pred_15,pred_150,pred_151,pred_152,pred_153,pred_154,pred_155,pred_156,pred_157,pred_158,pred_159,pred_16,pred_160,pred_161,pred_162,pred_163,pred_164,pred_165,pred_166,pred_167,pred_168,pred_169,pred_17,pred_170,pred_171,pred_172,pred_173,pred_174,pred_175,pred_176,pred_177,pred_178,pred_179,pred_18,pred_180,pred_181,pred_182,pred_183,pred_184,pred_185,pred_186,pred_187,pred_188,pred_189,pred_19,pred_190,pred_191,pred_192,pred_193,pred_194,pred_195,pred_196,pred_197,pred_198,pred_199,pred_2,pred_20,pred_200,pred_201,pred_202,pred_203,pred_204,pred_205,pred_206,pred_207,pred_208,pred_209,pred_21,pred_210,pred_211,pred_212,pred_213,pred_214,pred_215,pred_216,pred_217,pred_218,pred_219,pred_22,pred_220,pred_221,pred_222,pred_223,pred_224,pred_225,pred_226,pred_227,pred_228,pred_229,pred_23,pred_230,pred_231,pred_232,pred_233,pred_234,pred_235,pred_236,pred_237,pred_238,pred_239,pred_24,pred_240,pred_241,pred_242,pred_243,pred_244,pred_245,pred_246,pred_247,pred_248,pred_249,pred_25,pred_250,pred_251,pred_252,pred_253,pred_254,pred_255,pred_256,pred_257,pred_258,pred_259,pred_26,pred_260,pred_261,pred_262,pred_263,pred_264,pred_265,pred_266,pred_267,pred_268,pred_269,pred_27,pred_270,pred_271,pred_28,pred_29,pred_3,pred_30,pred_31,pred_32,pred_33,pred_34,pred_35,pred_36,pred_37,pred_38,pred_39,pred_4,pred_40,pred_41,pred_42,pred_43,pred_44,pred_45,pred_46,pred_47,pred_48,pred_49,pred_5,pred_50,pred_51,pred_52,pred_53,pred_54,pred_55,pred_56,pred_57,pred_58,pred_59,pred_6,pred_60,pred_61,pred_62,pred_63,pred_64,pred_65,pred_66,pred_67,pred_68,pred_69,pred_7,pred_70,pred_71,pred_72,pred_73,pred_74,pred_75,pred_76,pred_77,pred_78,pred_79,pred_8,pred_80,pred_81,pred_82,pred_83,pred_84,pred_85,pred_86,pred_87,pred_88,pred_89,pred_9,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99,pred_272,pred_273,pred_274,pred_275,pred_276,pred_277,t_mean,t_median,t_sum_log,t_sum_act,t_nb_sess,PredictedLogRevenue
fullVisitorId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000000259678714014,0.0,

In [ ]:
train.head()